In [9]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [36]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

In [183]:
job_104_url=  "https://www.104.com.tw/jobs/search/?ro=0&jobcat=2008001020%2C2008001016%2C2008001014%2C2008001015&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001006000&order=11&asc=0&page=1&mode=s&jobsource=tab_cs_to_job"

time_back_N_day = int(input("請輸入要查詢的天數:"))

#job_104_url_ECE = "https://www.104.com.tw/jobs/search/?ro=0&jobcat=2008001020%2C2008001016%2C2008001014&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001006000&order=11&asc=0&page=1&mode=s&jobsource=tab_cs_to_job" 

請輸入要查詢的天數:2


In [184]:
# 目前網站是 104人力銀行 已選取 新竹市 電信、類比IC、半導體工程師

browser = webdriver.Chrome()

browser.get(job_104_url)

within_N_day = time_back_N_day*24*60*60
time_back_N_day = time.localtime(time.time()-within_N_day)
i=0

while(1):
    
    html = browser.page_source
        
    
    soup = BeautifulSoup(html)
    
    company_list = soup.find("div", attrs={'id':"js-job-content"}).find_all('article')   ##constantly get the new data
    
    Date_Lastest_str = company_list[-1].span.text.replace(' ','').replace('\n','')  ###get the oldest time of the new data
    Date_Lastest_struct = time.strptime ( Date_Lastest_str   ,   "%m/%d"  )

    if (Date_Lastest_struct.tm_mday <= time_back_N_day.tm_mday):   ##   latest day may be N-1 or N 
        j=0
        for j in range( 0,len(company_list) ):
             #print(len(company_list))
            if len(company_list[j].span.text.replace(' ','').replace('\n','')) != 0:  ## 104前面幾筆資料可能會沒有日期
                #print(company_list[j].span.text.replace(' ','').replace('\n',''))
                company_time = time.strptime(  company_list[j].span.text.replace(' ','').replace('\n','')  ,     "%m/%d"  )
                print(j)
                j=j+1
                
                
                if(company_time.tm_mday ==time_back_N_day.tm_mday):
                    
                    break
        break
        
              
        
    
    browser.execute_script("window.scrollTo( 0 ,  document.body.scrollHeight  )")   ## scroll the height of a page
    try:
        if soup.find_all(class_="js-more-page") !=None:    ### 從點第一個開始，會有好個more-page
            browser.find_elements_by_class_name ('js-more-page')[i].click()  ## 必須與更下面的more-page互動，所以弄成list，再選
            i=i+1
            print('sucessful_click')
    except:   ##  因為一開始沒有可點選的按鈕，如果不加這個，會因為前面找不到按鈕程式就停下來
        print('exception') 
    
    time.sleep(2) ## delay 一段時間等網頁回應再解析，否則會當掉



## ----------------------------------------Main --------------------------------
k=0

company_list_TimeMatched = []  ##  重建一個符合時間的 company_list
#print(j)  ##這邊有點奇怪，不知道為甚麼break 之後 j自動+1了
for k in range(0,j-1):   #  j 是while迴圈break前符合需要時間的所有公司數量
    company_list_TimeMatched.append(company_list[k])
    k=k+1
#print(company_list_TimeMatched)
company_name_address=[]  ##因爬取到的string有包含公司名子和住址，且是以跳行做 字的分隔，所以我先放在一個list裡，之後再用split分開list的元素
company_name=[]
company_address=[]
job_title=[]
job_class=[]
job_link=[]
job_content=[]
job_salary=[]
i=-1
for d in company_list_TimeMatched:
    
    #print("----------------------------------------------------------------------")
    # 公司名稱與地址
    #company_name = d.find_all('li')[1].a['title']
    #print(company_name)
    
    company_name_address.append(d.find_all('li')[1].a['title'])
    i=i+1
    
    company_name.append(company_name_address[i].split('\n')[0].replace('公司名：','')) 
    company_address.append(company_name_address[i].split('\n')[1].replace('公司住址：','').replace('公司地址：',''))
        
        
    '''  
    Why?company_name_address[i].split('\n')[0]:  因為company_name_address中含有很多個list，不能直接分割，分割後的第一個元素是名子
    Why?replace:  因為稍後會放入dict，所以不需要多餘的字
    ''' 
    
    
    #職稱
    #job_title = d.a.text
    #print(job_title)
    job_title.append(d.a.text) 
    
    # 職務類別
    #job_class = "職務類別:"+d.find_all('li')[2].text
    #print(job_class)
    job_class.append(d.find_all('li')[2].text)
    
    # 網址
    #job_link = "104應徵網站:" + " " + "https:"+ d.a['href']
    #print(job_link)
    job_link.append("https:"+ d.a['href'])
    
    # 工作內容
    #job_content = d.p.text
    #print(job_content)
    job_content.append(d.p.text)
    
    # 薪資
    #job_salary = "薪資:" + d.find('span' , attrs = {'class' , 'b-tag--default'}).text
    #print(job_salary)
    job_salary.append(d.find('span' , attrs = {'class' , 'b-tag--default'}).text)
#for j in range(0,722+1):  
Sum={"公司名稱":company_name,"地址":company_address,"職稱":job_title,"職務類別":job_class,"網址":job_link,"薪資":job_salary,"工作內容":job_content}
df=pd.DataFrame(Sum)
df    


2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
10
11
12
13


,公司名稱,地址,職稱,職務類別,網址,薪資,工作內容
0,旺宏電子股份有限公司,新竹市力行路16號,研發類 - 製程整合研發(Process Integration R&D) 工程師(ME000),新竹市,https://www.104.com.tw/job/7ajmd?jobsource=hot...,待遇面議,1.Integrate fabrication processes of PCM memor...
1,旺宏電子股份有限公司,新竹市力行路16號,IC設計類 - 數位/類比IC設計工程師,新竹市,https://www.104.com.tw/job/5q1va?jobsource=hot...,待遇面議,1.類比/數位電路設計研發、模擬與驗證、產品驗證。\n2.數位IC設計需具Verilog技能。
2,挪威商諾迪克半導體股份有限公司台灣分公司,新竹縣竹北市,Test Engineer,IC設計相關業,https://www.104.com.tw/job/6lqmz?jobsource=tab...,待遇面議,You will be responsible for quality and timeli...
3,軒宙科技有限公司,新竹市中華路一段195號,監視系統設備安裝工程師,其他電信及通訊相關業,https://www.104.com.tw/job/6jdid?jobsource=tab...,"月薪32,000~40,000元","1.主要服務客戶：政府單位的監視器檢測,裝配與調整各項監控產品或設備。 \n2.負責更換、調..."
4,美商賽發馥股份有限公司臺灣分公司,新竹市關新路27號5樓之5,Design Verification Engineer,電腦軟體服務業,https://www.104.com.tw/job/6r4sl?jobsource=tab...,待遇面議,SiFive is an idea-to-silicon company founded b...
5,美商賽發馥股份有限公司臺灣分公司,新竹市關新路27號5樓之5,RISC-V FPGA Verification Engineer,電腦軟體服務業,https://www.104.com.tw/job/713af?jobsource=tab...,待遇面議,SiFive is an idea-to-silicon company founded b...
6,美商賽發馥股份有限公司臺灣分公司,新竹市關新路27號5樓之5,Software Engineer for RISC-V Vector,電腦軟體服務業,https://www.104.com.tw/job/74gxd?jobsource=tab...,待遇面議,SiFive is an idea-to-silicon company founded b...
7,美商賽發馥股份有限公司臺灣分公司,新竹市,RTL Design Engineer - RISC-V CPU,電腦軟體服務業,https://www.104.com.tw/job/6tpst?jobsource=tab...,待遇面議,SiFive is an idea-to-silicon company founded b...
8,挪威商諾迪克半導體股份有限公司台灣分公司,新竹縣竹北市,Staff Test Engineer,IC設計相關業,https://www.104.com.tw/job/6lr7y?jobsource=tab...,待遇面議,"As a Staff Test Engineer, you will assume tech..."
9,美商賽發馥股份有限公司臺灣分公司,新竹市,Senior RTL Design Engineer - RISC-V CPU,電腦軟體服務業,https://www.104.com.tw/job/74zup?jobsource=tab...,待遇面議,SiFive is an idea-to-silicon company founded b...


In [4]:
df.to_excel("output.xlsx")  

In [41]:
for d in company_list:
    #print("----------------------------------------------------------------------")
    # 公司名稱與地址
    company_name = d.find_all('li')[1].a['title']
    print(company_name)
    #company_name_address.append(d.find_all('li')[1].a['title'])
    
    #職稱
    job_title = d.a.text
    print(job_title)
    #job_title.append(d.a.text) 
    
    # 職務類別
    job_class = "職務類別:"+d.find_all('li')[2].text
    print(job_class)
    #job_class.append(d.find_all('li')[2].text)
    
    # 網址
    job_link = "104應徵網站:" + " " + "https:"+ d.a['href']
    print(job_link)
    #job_link.append("https:"+ d.a['href'])
    
    # 工作內容
    job_content = d.p.text
    print(job_content)
    #job_content.append(d.p.text)
    
    # 薪資
    job_salary = "薪資:" + d.find('span' , attrs = {'class' , 'b-tag--default'}).text
    print(job_salary)
    #job_salary.append(d.find('span' , attrs = {'class' , 'b-tag--default'}).text)

公司名：旺宏電子股份有限公司
公司住址：新竹市力行路16號
IC設計類 - 數位/類比IC設計工程師
職務類別:新竹市
104應徵網站: https://www.104.com.tw/job/5q1va?jobsource=hotjob_chr
1.類比/數位電路設計研發、模擬與驗證、產品驗證。
2.數位IC設計需具Verilog技能。
薪資:待遇面議
公司名：瑞昱半導體股份有限公司
公司住址：新竹市科學園區創新二路2號
類比IC設計工程師R7
職務類別:新竹市
104應徵網站: https://www.104.com.tw/job/65p58?jobsource=hotjob_chr
工作項目：
1. 投入wifi 11ax類比電路開發工作。
2. 投入10G Base-T及2.5G Base-T類比電路開發工作。
3. 投入Long Distance Ethernet類比電路開發工作。
4. 投入Giga automotive類比電路開發工作。

應徵條件：
1. 碩士以上; 電機工程、電信工程、電控工程、電子工程相關科系畢業為主。
2. 有 ADC/DAC/PLL經驗尤佳。
薪資:待遇面議
公司名：旺宏電子股份有限公司
公司住址：新竹市力行路16號
研發類 - 製程整合研發(Process Integration R&D) 工程師(ME000)
職務類別:新竹市
104應徵網站: https://www.104.com.tw/job/7ajmd?jobsource=hotjob_chr
1.Integrate fabrication processes of PCM memory in a research lab. for SCM and analog AI applications.
2.Prepare basic samples and analyze data from analysis techniques such as top-down SEM, cross-sectional SEM, TEM, EDS and SIMS.  
3.Cooperate with researchers to evaluate new materials, new process and new tools that help memory and analog 

薪資:待遇面議
公司名：光環科技股份有限公司 
公司地址：新竹市展業一路21號
【110年研發替代役】長波雷射元件研發工程師
職務類別:光電產業
104應徵網站: https://www.104.com.tw/job/73vob?jobsource=tab_cs_to_job
10G、25G長波雷射元件產品開發及特性驗證


薪資:待遇面議
公司名：宏觀微電子股份有限公司 
公司地址：新竹縣竹北市成功十二街28號8樓
類比IC設計(PLL)工程師專案經理
職務類別:IC設計相關業
104應徵網站: https://www.104.com.tw/job/5wicj?jobsource=tab_cs_to_job
隨著5G勢如破竹，帶動多元應用與智慧移動需求的竄升，作為關鍵一環的無線射頻元件，也將因為無線通訊模組技術的急速升級而更加重要.

工作任務：無線射頻混合訊號Mixed-Signal Integrated Circuit 應用
需求條件：
1.具備3年以上 mixed signal 或 RF 相關電路設計經驗.
2.熟悉 phase locked loop,frequency synthesizer and LO design.
3.熟悉 Cadence 設計軟體及半導體製程. 
4.具備通訊基本知識.
薪資:待遇面議
公司名：美商超微半導體股份有限公司台灣分公司 
公司地址：新竹市公道五路二段101號8樓
IP architect
職務類別:半導體製造業
104應徵網站: https://www.104.com.tw/job/74wjc?jobsource=tab_cs_to_job
What you do at AMD changes everything
At AMD, we push the boundaries of what is possible.  We believe in changing the world for the better by driving innovation in high-performance computing, graphics, and visualization technologies – building blocks for gaming, Immersive platforms, and the d

工作項目：
1、ADC design
2、DAC, TX class AB output, line driver design
3、Automotive IP development
4、Automotive project leader
薪資:待遇面議
公司名：瑞昱半導體股份有限公司 
公司地址：新竹市新竹科學園區創新二路2號
BT Audio SoC系統設計工程師
職務類別:半導體製造業
104應徵網站: https://www.104.com.tw/job/6o0tf?jobsource=tab_cs_to_job
工作項目：
1. BT SoC PMU IC verification.
2. BT SoC Codec IC verification.
3. BT Soc CP/FT/reliability test.

應徵條件：
1. 碩士以上; 電機工程、電信工程、電控工程、電子工程相關科系畢業為主。
薪資:待遇面議
公司名：創意電子股份有限公司 
公司地址：新竹市新竹科學園區力行六路10號
Execution Project Manager / General Solution Architect (竹科) ("享自我推薦獎金，獎金最高15萬”)
職務類別:IC設計相關業
104應徵網站: https://www.104.com.tw/job/73vyj?jobsource=tab_cs_to_job
※Job Contents:
1. 工作地點：Hsinchu 
2. Responsible for ASIC tech activities of pre-sale projects, including tech capability promotion, Effective communication with clients for tech and business needs, tech proposal delivery and SOW drafting.
3. Handle ASIC program management and key issue resolving from Netlist-in to Mass production.
4. Business travel is expected. 

※Requirements:


In [ ]:
## https://kknews.cc/zh-tw/code/a3me3ln.html 